In [1]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [2]:
class AllDSVTBlocksTRT(nn.Module):
    def __init__(self, dsvtblocks_list, layer_norms_list):
        super().__init__()
        self.layer_norms_list = layer_norms_list
        self.dsvtblocks_list = dsvtblocks_list
    def forward(
        self,
        pillar_features,
        set_voxel_inds_tensor_shift_0,
        set_voxel_inds_tensor_shift_1,
        set_voxel_masks_tensor_shift_0,
        set_voxel_masks_tensor_shift_1,
        pos_embed_tensor,
    ):
        outputs = pillar_features

        residual = outputs
        blc_id = 0
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 1
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)

        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 2
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_0[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_0[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        residual = outputs
        blc_id = 3
        set_id = 0
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)
        set_id = 1
        set_voxel_inds = set_voxel_inds_tensor_shift_1[set_id:set_id+1].squeeze(0)
        set_voxel_masks = set_voxel_masks_tensor_shift_1[set_id:set_id+1].squeeze(0)
        pos_embed = pos_embed_tensor[blc_id:blc_id+1, set_id:set_id+1].squeeze(0).squeeze(0)
        inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed)
        # inputs = (outputs, set_voxel_inds, set_voxel_masks, pos_embed, True)
        outputs = self.dsvtblocks_list[blc_id].encoder_list[set_id](*inputs)

        outputs = self.layer_norms_list[blc_id](residual + outputs)

        return outputs

In [5]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_3d')==False:
    os.mkdir('./deploy_pillar_sfaw_3d')
log_file = './deploy_pillar_sfaw_3d/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-04-11 05:15:29,352   INFO  Loading Custom dataset.
2024-04-11 05:15:29,352   INFO  Loading Custom dataset.
2024-04-11 05:15:29,355   INFO  Total samples for CUSTOM dataset: 0
2024-04-11 05:15:29,355   INFO  Total samples for CUSTOM dataset: 0


In [6]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
# cfg

In [7]:
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-04-11 05:15:32,371   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-11 05:15:32,371   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-11 05:15:32,884   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-11 05:15:32,884   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-11 05:15:32,929   INFO  ==> Done (loaded 391/391)
2024-04-11 05:15:32,929   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [8]:
# batch_dict = torch.load("./batch_dict.pth", map_location="cuda")
# inputs = batch_dict
# inputs

In [9]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")

inputs = batch_dict
inputs

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1}

In [10]:
with torch.no_grad():
    DSVT_Backbone = model.backbone_3d
    dsvtblocks_list = DSVT_Backbone.stage_0
    layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    inputs = model.vfe(inputs)
    voxel_info = DSVT_Backbone.input_layer(inputs)
    set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]

    pillar_features = inputs['voxel_features']
    alldsvtblockstrt_inputs = (
        pillar_features,
        set_voxel_inds_list[0][0],
        set_voxel_inds_list[0][1],
        set_voxel_masks_list[0][0],
        set_voxel_masks_list[0][1],
        torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0),
    )

In [15]:
inputs

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1,
 'pillar_features': tensor([[0.0000, 0.0458, 0.0000,  ..., 0.0303, 0.0000, 0.0000],
         [0.0000, 0.0686, 0.0000,  ..., 0.0079, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.5312, 0.4485, 0.0000,  ..., 0.0691, 0.0000, 0.5523],
         [0.0000, 0.4938, 0.0000,  ..., 0.0557, 0.0

In [11]:

jit_mode = "trace"
input_names = [
    'src',
    'set_voxel_inds_tensor_shift_0',
    'set_voxel_inds_tensor_shift_1',
    'set_voxel_masks_tensor_shift_0',
    'set_voxel_masks_tensor_shift_1',
    'pos_embed_tensor'
]
output_names = ["output",]
input_shapes = {
    "src": {
        "min_shape": [24629, 192],
        "opt_shape": [24629, 192],
        "max_shape": [24629, 192],
    },
    "set_voxel_inds_tensor_shift_0": {
        "min_shape": [2, 1156, 36],
        "opt_shape": [2, 1156, 36],
        "max_shape": [2, 1156, 36],
    },
    "set_voxel_inds_tensor_shift_1": {
        "min_shape": [2, 834, 36],
        "opt_shape": [2, 834, 36],
        "max_shape": [2, 834, 36],
    },
    "set_voxel_masks_tensor_shift_0": {
        "min_shape": [2, 1156, 36],
        "opt_shape": [2, 1156, 36],
        "max_shape": [2, 1156, 36],
    },
    "set_voxel_masks_tensor_shift_1": {
        "min_shape": [2, 834, 36],
        "opt_shape": [2, 834, 36],
        "max_shape": [2, 834, 36],
    },
    "pos_embed_tensor": {
        "min_shape": [4, 2, 24629, 192],
        "opt_shape": [4, 2, 24629, 192],
        "max_shape": [4, 2, 24629, 192],
    },
}

dynamic_axes = {
    "src": {
        0: "voxel_number",
    },
    "set_voxel_inds_tensor_shift_0": {
        1: "set_number_shift_0",
    },
    "set_voxel_inds_tensor_shift_1": {
        1: "set_number_shift_1",
    },
    "set_voxel_masks_tensor_shift_0": {
        1: "set_number_shift_0",
    },
    "set_voxel_masks_tensor_shift_1": {
        1: "set_number_shift_1",
    },
    "pos_embed_tensor": {
        2: "voxel_number",
    },
    "output": {
        0: "voxel_number",
    }
}

In [12]:
base_name = "./deploy_pillar_sfaw_3d/dsvt_3d_bb"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    allptransblocktrt,
    alldsvtblockstrt_inputs,
    onnx_path, input_names=input_names,
    output_names=output_names, dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [13]:
# test onnx
ort_session = ort.InferenceSession(onnx_path)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
              ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
              ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
              ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
              ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
              ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0].shape


(962, 192)

In [14]:
ort_outs

[array([[-1.09687078e+00, -8.87880176e-02,  1.56932533e-01, ...,
         -1.04772127e+00,  3.79999131e-02, -1.98598072e-01],
        [-1.09640002e+00, -7.08313286e-02,  1.60522014e-01, ...,
         -1.08235109e+00,  4.23637107e-02, -1.92351446e-01],
        [-1.08620560e+00, -1.00942045e-01,  1.78571314e-01, ...,
         -1.05504239e+00,  2.63126697e-02, -1.59484476e-01],
        ...,
        [-3.60771596e-01,  2.12829709e-01, -1.58340335e-01, ...,
         -1.23321140e+00, -1.63467094e-01,  1.67723313e-01],
        [-6.18591845e-01,  3.51619005e-01, -1.11299694e-01, ...,
         -1.37231791e+00, -1.09308995e-01, -5.45044728e-02],
        [-6.13398790e-01,  3.21275651e-01, -1.22285048e-02, ...,
         -1.24491286e+00,  6.93764538e-04, -1.37991160e-01]], dtype=float32)]